<a href="https://colab.research.google.com/github/ATdigitalhumanist/DeepLearning/blob/main/DL_Assignment_Part_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part 3: Image Classification using Transfer Learning
**Objective**: The objective of this assignment is to develop an image classification model using transfer learning. You will find a pretrain a neural network model, and then use transfer learning to classify logos of popular food chains.

In [ ]:
# for System library
import os
import random
from io import BytesIO

# for numerical processing library
import numpy as np

# for tensorflow and keras processing librarcy
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16  # You can change this to ResNet or another model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input

# for spliting dataset into train and test
from sklearn.model_selection import train_test_split

# for image loading and proecssing
import cv2
import glob

In [ ]:
# Set random seed for reproducibility
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
# Mount Google Drive in the Colab notebook
from google.colab import drive
drive.mount('/content/gdrive')

Load Training Data from Logos3 folder

In [ ]:
baseDir = "/content/gdrive/MyDrive/DL Assignment/Part 3/logos3/train"
testDir = "/content/gdrive/MyDrive/DL Assignment/Part 3/logos3/test"

Name=[]

for file in os.listdir(baseDir):
  Name+=[file]

print(Name)
print(len(Name))

brandMap = dict(zip(Name, [t for t in range(len(Name))]))
print(brandMap)
rBrandMap=dict(zip([t for t in range(len(Name))],Name))

testName=[]

for file in os.listdir(testDir):
  testName+=[file]

print(testName)
print(len(testName))

brandMapTest = dict(zip(Name, [t for t in range(len(Name))]))
print(brandMapTest)
rBrandMapTest = dict(zip([t for t in range(len(Name))],Name))

In [ ]:
import matplotlib.pyplot as plt
Brand = '/content/gdrive/MyDrive/DL Assignment/Part 3/logos3/train/Starbucks'
import os
subClass = os.listdir(Brand)

fig = plt.figure(figsize = (10,5))
for e in range(len(subClass[:10])):
  plt.subplot(2,5,e+1)
  img = plt.imread(os.path.join(Brand, subClass[e]))
  plt.imshow(img, cmap=plt.get_cmap('gray'))
  plt.axis('off')

In [ ]:
# Define image dimensions and batch size
img_height, img_width = 224, 224
batch_size = 32

Generate Train data loaded from Train Directory. Create Train and Validation Data from the Training Data loaded. Validation data is 20% of Train data.

In [ ]:
# Create data generators with data augmentation for training data
trainDataGen = ImageDataGenerator(
    rescale=1.0 / 255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

In [ ]:
# Create data generators for validation data
validDataGen = ImageDataGenerator(rescale=1.0 / 255.0, validation_split=0.2)

In [ ]:
# Generate training data from the images in the train directory
trainGenerator = trainDataGen.flow_from_directory(
    baseDir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

In [ ]:
# Generate validation data from the images in the train directory
validGenerator = validDataGen.flow_from_directory(
    baseDir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

Generate Test data from the images loaded from the Test directory

In [ ]:
# Generate test data from the images in the test directory
testDataGen = ImageDataGenerator(rescale=1.0 / 255.0)
testGenerator = testDataGen.flow_from_directory(
    testDir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

Define Base Model for VGG-16

In [ ]:
# Load the pre-trained VGG-16 model
baseModelVGG16 = VGG16(
    weights='imagenet',
    include_top=False,
    input_shape=(img_height, img_width, 3)
)
# Freeze the layers of the pre-trained model
for layer in baseModelVGG16.layers:
    layer.trainable = False

In [ ]:
from tensorflow.keras.layers import Conv2D, Flatten, Dense, Dropout

# Create a new model on top of the pre-trained model
model = Sequential([
    baseModelVGG16,
    Conv2D(256, (3, 3), activation='relu', padding='same'),
    Conv2D(256, (3, 3), activation='relu', padding='same'),

    Conv2D(512, (3, 3), activation='relu', padding='same'),
    Conv2D(512, (3, 3), activation='relu', padding='same'),

    Conv2D(1024, (3, 3), activation='relu', padding='same'),
    Conv2D(1024, (3, 3), activation='relu', padding='same'),

    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(trainGenerator.num_classes, activation='softmax')  # Number of classes determined by the training generator
])

In [ ]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Print model summary
model.summary()

In [ ]:
# Replace the last fully connected layer
numClasses = len(trainGenerator.class_indices)

# Create a new fully connected layer with the appropriate number of output nodes
newFCLayer = Dense(numClasses, activation='softmax')

# Replace the last layer of the pre-trained model with the new fully connected layer
model.layers[-1] = newFCLayer

In [ ]:
# Freeze the weights of the pre-trained layers
for layer in baseModelVGG16.layers:
    layer.trainable = False

# Compile the model again after modifying it
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Train only the new fully connected layer using the training set
epochsFineTune = 10  # Adjust the number of epochs for fine-tuning
historyFineTune = model.fit(
    trainGenerator,
    steps_per_epoch=trainGenerator.samples // batch_size,
    validation_data=validGenerator,
    validation_steps=validGenerator.samples // batch_size,
    epochs=epochsFineTune
)

In [ ]:
# Evaluate the performance of the model using the validation set before fine-tuning
valLossFineTune, valAccuracyFineTune = model.evaluate(validGenerator)
print("Validation Loss before Fine-Tuning:", valLossFineTune)
print("Validation Accuracy before Fine-Tuning:", valAccuracyFineTune)

In [ ]:
plt.plot(historyFineTune.history['accuracy'])
plt.plot(historyFineTune.history['val_accuracy'])
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.show()

training_accuracy = historyFineTune.history['loss']
validation_accuracy = historyFineTune.history['val_loss']
plt.plot(training_accuracy, 'r', label = 'training loss')
plt.plot(validation_accuracy, 'b', label = 'validation loss')
plt.title('Training and Validation Loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()
plt.show()

In [ ]:
# Evaluate the model on the test data
testLoss, testAccuracy = model.evaluate(testGenerator)
print("Test Loss:", testLoss)
print("Test Accuracy:", testAccuracy)

In [ ]:
# Unfreeze some of the pre-trained layers for further fine-tuning
# For example, you can unfreeze the last few convolutional layers:
for layer in baseModelVGG16.layers[-10:]:
    layer.trainable = True

# Recompile the model after unfreezing some layers
model.compile(optimizer=Adam(learning_rate=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Evaluate the performance of the model using the validation set after further fine-tuning
epochsFinalFineTune = 10  # Adjust the number of epochs for final fine-tuning
historyFinalFineTune = model.fit(
    trainGenerator,
    steps_per_epoch=trainGenerator.samples // batch_size,
    validation_data=validGenerator,
    validation_steps=validGenerator.samples // batch_size,
    epochs=epochsFinalFineTune
)

In [ ]:
# Evaluate the final fine-tuned model on the validation set after fine tunning the model
valLossFinalFineTune, valAccuracyFinalFineTune = model.evaluate(validGenerator)
print("Validation Loss after Final Fine-Tuning:", valLossFinalFineTune)
print("Validation Accuracy after Final Fine-Tuning:", valAccuracyFinalFineTune)

In [ ]:
plt.plot(historyFineTune.history['accuracy'])
plt.plot(historyFineTune.history['val_accuracy'])
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Validation accuracy after fine tunning')
plt.legend()
plt.show()

training_accuracy = historyFineTune.history['loss']
validation_accuracy = historyFineTune.history['val_loss']
plt.plot(training_accuracy, 'r', label = 'training loss')
plt.plot(validation_accuracy, 'b', label = 'validation loss')
plt.title('Training and Validation loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()
plt.show()

In [ ]:
# Evaluate the model on the test data after fine tunning the model
testLoss, testAccuracy = model.evaluate(testGenerator)
print("Test Loss:", testLoss)
print("Test Accuracy:", testAccuracy)

In [ ]:
# Print the names of all layers in the model
for layer in model.layers:
    print(layer.name)

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from io import BytesIO
import numpy as np
import matplotlib.pyplot as plt

# Choose a layer from which you want to visualize the feature maps
layerName = 'conv2d_1'

# Create a sub-model to extract feature maps from the chosen layer
visualizationModel = Model(inputs=model.input, outputs=model.get_layer(layerName).output)

# Choose an example image for visualization (use any image from your dataset)
from google.colab import drive
drive.mount('/content/gdrive')

from google.colab import files
uploadedFiles = files.upload()

exampleImageData = uploadedFiles['BKImage3.jpg']

# Convert the image data to a BytesIO object
exampleImageBytes = BytesIO(exampleImageData)

exampleImage = image.load_img(exampleImageBytes, target_size=(224, 224))
exampleImage = image.img_to_array(exampleImage)
exampleImage = np.expand_dims(exampleImage, axis=0)
exampleImage = preprocess_input(exampleImage)

# Get the feature maps for the example image
featureMaps = visualizationModel.predict(exampleImage)

# Number of feature maps in the chosen layer
numFeatureMaps = featureMaps.shape[-1]

# Define the grid dimensions for plotting
gridSize = int(np.ceil(np.sqrt(numFeatureMaps)))

# Create a figure and axis for plotting
fig, ax = plt.subplots(gridSize, gridSize, figsize=(10, 10))

# Plot each feature map
for i in range(numFeatureMaps):
    row = i // gridSize
    col = i % gridSize
    ax[row, col].imshow(featureMaps[0, :, :, i], cmap='viridis')
    ax[row, col].axis('off')

# Display the uploaded image name as a title
plt.suptitle('Uploaded Image: BKImage3.jpg', fontsize=16)

# Show the plots
plt.show()
